##Exemplo Classificador de Flores - TensorFlow
 

(iris.jpg)

In [1]:
import tensorflow as tf
import pandas as pd

COLUMN_NAMES = [
        'SepalLength', 
        'SepalWidth',
        'PetalLength', 
        'PetalWidth', 
        'Species'
        ]

# Importando dataset de treinamento
training_dataset = pd.read_csv('iris_training.csv', names=COLUMN_NAMES, header=0)
train_x = training_dataset.iloc[:, :4]
train_y = training_dataset.iloc[:, 4]

# Importando dataset de teste
test_dataset = pd.read_csv('iris_test.csv', names=COLUMN_NAMES, header=0)
test_x = test_dataset.iloc[:, :4]
test_y = test_dataset.iloc[:, 4]


/Users/lucasosouza/anaconda/envs/udacity/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


(iris_dataset.png)

In [2]:
tf.__version__

'1.5.0'

In [3]:
# Agora, precisamos definir colunas de recursos, que irão ajudar a nossa Rede Neural.

columns_feat = [
    tf.feature_column.numeric_column(key='SepalLength'),
    tf.feature_column.numeric_column(key='SepalWidth'),
    tf.feature_column.numeric_column(key='PetalLength'),
    tf.feature_column.numeric_column(key='PetalWidth')
]

(iris_modelo.png)

In [19]:
#Construindo a Rede Neural - DNNClassifier

classifier = tf.estimator.DNNClassifier(

    feature_columns=columns_feat,

    # 2 camadas ocultas com 10 nós cada
    hidden_units=[10, 10],

    # O modelo está classificando 3 classes
    n_classes=3)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1311266a0>, '_tf_random_seed': None, '_task_id': 0, '_num_worker_replicas': 1, '_save_summary_steps': 100, '_session_config': None, '_service': None, '_task_type': 'worker', '_is_chief': True, '_save_checkpoints_secs': 600, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/var/folders/dl/zfkdh1395z33j_svnym2xm_00000gn/T/tmp4nad7zpv', '_num_ps_replicas': 0, '_master': '', '_log_step_count_steps': 100, '_save_checkpoints_steps': None}


In [16]:
help(tf.data.Dataset)

Help on class Dataset in module tensorflow.python.data.ops.dataset_ops:

class Dataset(builtins.object)
 |  Represents a potentially large set of elements.
 |  
 |  A `Dataset` can be used to represent an input pipeline as a
 |  collection of elements (nested structures of tensors) and a "logical
 |  plan" of transformations that act on those elements.
 |  
 |  Methods defined here:
 |  
 |  __init__(self)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  apply(self, transformation_func)
 |      Apply a transformation function to this dataset.
 |      
 |      `apply` enables chaining of custom `Dataset` transformations, which are
 |      represented as functions that take one `Dataset` argument and return a
 |      transformed `Dataset`.
 |      
 |      For example:
 |      
 |      ```
 |      dataset = (dataset.map(lambda x: x ** 2)
 |                 .apply(group_by_window(key_func, reduce_func, windo

In [20]:
#Definindo função de treinamento

def train_function(inputs, outputs, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(inputs), outputs))
    print(dir(dataset))
    dataset = dataset.shuffle(1000).repeat().batch(batch_size)
    return dataset #.make_one_shot_iterator().get_next()

#Treinar o Modelo

classifier.train(
    input_fn=lambda:train_function(train_x, train_y, 100),
    steps=1000)


['_GeneratorState', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_as_variant_tensor', '_enumerate', '_output_classes', '_output_shapes', '_output_types', '_tensors', 'apply', 'batch', 'cache', 'concatenate', 'filter', 'flat_map', 'from_generator', 'from_sparse_tensor_slices', 'from_tensor_slices', 'from_tensors', 'interleave', 'list_files', 'make_initializable_iterator', 'make_one_shot_iterator', 'map', 'output_classes', 'output_shapes', 'output_types', 'padded_batch', 'prefetch', 'range', 'repeat', 'shard', 'shuffle', 'skip', 'take', 'zip']
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/dl/zfkdh1395z33j_svnym2xm_00000gn/T/tmp4nad7zpv/model

In [21]:
#Definindo função de avaliação
def evaluation_function(attributes, classes, batch_size):
    attributes=dict(attributes)
    if classes is None:
        inputs = attributes
    else:
        inputs = (attributes, classes)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)
    return dataset # .make_one_shot_iterator().get_next()

# Avaliando o classificador
eval_result = classifier.evaluate(
    input_fn=lambda:evaluation_function(test_x, test_y, 100))

INFO:tensorflow:Starting evaluation at 2018-02-06-17:00:30
INFO:tensorflow:Restoring parameters from /var/folders/dl/zfkdh1395z33j_svnym2xm_00000gn/T/tmp4nad7zpv/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2018-02-06-17:00:30
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.96666664, average_loss = 0.056468733, global_step = 1000, loss = 1.694062
